## Retrieval Augmented Generation (RAG), with friends dataset

#### Loading friends dataset and creating vector database with 70% of the data (the rest is for testing later)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import chromadb

# loading friends data. this dataset is already cleaned from unessesary colunms and nan values
data = pd.read_csv('clean_friends.csv')

# "only use 1 procent of the data fast debugging
# data = data.sample(frac=0.01, random_state=42)

# ensure all data['text'] are strings
data['text'] = data['text'].astype(str)

# split the data into two parts: one for the vector database and one for testing
data_vector_db, data_rag_test = train_test_split(data, test_size=0.3, random_state=42)

# initialize Chroma client and create a collection
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="friends_collection")

# prepare the metadata/labels
labels = []
for label in data_vector_db['label']:
    labels.append({'label': label})

# add text documents to the collection with metadata and ids
collection.add(
    documents=data_vector_db['text'].tolist(), # e.g. ['jag blir mobbad', ... , 'klassen är stökig.']
    metadatas=labels, # e.g. [{'label': 0.0}, {'label': 1.0}, ... , {'label': 1.0}]
    ids=data_vector_db.index.astype(str).tolist() #  e.g.  ['3614', '2325', '1032', '2323']
)

# takes aprox 2.5min to run 

d:\ludvi\Documents\exjobb\kod\BullyingDetection\.conda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Prompting LLama 3 8B run LM-studio
##### The texts from test data are prompted to Llama 3, one at a time. 
##### Each prompt includes the 6 most similar texts from the vector database(3 of each label).

##### Example prompt:

```
Klassificera den sista textmeningen utifrån om personen som skrivit
den verkar mobbad i skolmiljön. Om det tyder på mobbning, svara
endast med ’1.0 ’. Annars , svara endast med ’0.0 ’. Klassificeringen
äller enbart om författaren av texten är mobbad. Innehåll som bara
är stötande eller våldsamt innebär inte automatiskt att texten ska
märkas som ’1.0 ’.

Här är sex märkta exempel :

- " Folk brukar kasta saker på mig " - label : 1.0
- " Lärarna brukar spotta på mig " - label : 1.0
- " Folk brukar kalla mig fula saker osv." - label : 1.0
- " Alla brukar sjunga ramsor om mig " - label : 0.0
- " Mina klasskamrater försöker mata mig" - label : 0.0
- " Vissa brukar jaga mig " - label : 0.0

Klassifiera nu följande text : "klasskamraterna brukar slå mig."

( OBS ! svara endast med ’1.0 ’ eller 0.0)

```

In [3]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# to store predictions and actual labels
predictions = []
actual_labels = []

# query each text from data_rag_test and classify
for count, (index, row) in enumerate(data_rag_test.iterrows(), 1):
    print('\n' * 10 + '-' * 300)
    print(f" Processing {count}/{len(data_rag_test)}...")  # indicates progress
   
    text_to_classify = row['text']
    actual_labels.append(row['label'])
   
    # querying the collection for the most similar 100 texts
    most_similar_collection = collection.query(
        query_texts=[text_to_classify],
        n_results=100

    )

    # extract texts, labels, and distances
    texts = most_similar_collection['documents'][0] 
    labels = most_similar_collection['metadatas'][0]
    
    # organize results by label, collect the first 3 closest matches for each label
    results_by_label = {1: [], 0: []}
    for i in range(len(texts)):
        label = labels[i]['label']
        if len(results_by_label[label]) < 3:
            results_by_label[label].append(texts[i])
            if len(results_by_label[0]) == 3 and len(results_by_label[1]) == 3:
                break  # stop once we have 3 texts for each label


    # prepare a string of example texts for the prompt
    similar_texts_string = ""
    for label in results_by_label:
        for text in results_by_label[label]:
            similar_texts_string += f"- {text}, label: {label}\n"


    # make RAG prediction
    valid_response = False
    while not valid_response:

        system_prompt = f"""\nKlassificera den sista textmeningen utifrån om personen som skrivit den verkar mobbad i skolmiljön. Om det tyder på mobbning, svara endast med '1.0'. Annars, svara endast med '0.0'. Klassificeringen gäller enbart om författaren av texten är mobbad. Innehåll som bara är stötande eller våldsamt innebär inte automatiskt att texten ska märkas som '1.0'.\n\nHär är sex märkta exempel:\n{similar_texts_string}\n\nKlassifiera texten i user prompten: """
        user_prompt = f"{text_to_classify}\n\n(OBS! svara endast '1.0' eller '0.0', inget annat!)"
        # print("\nPROMPT TO LLAMA 3:\n" + '- ' * 300 + system_prompt + user_prompt + '\n' + '- ' * 300)
            
        completion = client.chat.completions.create(
            model="lmstudio-community/Meta-Llama-3-8B-Instruct-GGUF",
            messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
            ],
            temperature=0.7
        )
        print("\n - LLama response:  " + completion.choices[0].message.content)

        response = completion.choices[0].message.content.strip()
        if response == '1.0' or response == '0.0':
            valid_response = True
            predicted_label = float(response)
            predictions.append(predicted_label)
            actual_label = row['label']
            print(" - Actual Label: ", actual_label)  
        
        print('-' * 300)













------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Processing 1/1123...

 - LLama response:  1.0
 - Actual Label:  0.0
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------










------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Process

## Results

In [5]:
from sklearn.metrics import classification_report, confusion_matrix, fbeta_score

# evaluate the model
report = classification_report(actual_labels, predictions, target_names=['Not Bullying', '    Bullying'])
f3_score_bullying = fbeta_score(actual_labels, predictions, beta=3, pos_label=1)
f3_score_non_bullying = fbeta_score(actual_labels, predictions, beta=3, pos_label=0)
f3_score_macro = (f3_score_bullying + f3_score_non_bullying) / 2

print(report)
print(f"F3 Score for Bullying: {f3_score_bullying}")
print(f"F3 Score for Non-Bullying: {f3_score_non_bullying}")
print(f"F3 Score Macro: {f3_score_macro}")

# evaluate the model using a confusion matrix
conf_matrix = confusion_matrix(actual_labels, predictions)
tn, fp, fn, tp = conf_matrix.ravel()

# less confusing explanation
print(f"\n  {tn + fp + fn + tp} cases in total in test set.")
print(f"\n  {tp} out of {tp + fn} actual bullying cases identified (true Positives).")
print(f"  {fp} false positives (non-bullying identified as bullying).")
print(f"  {fn} bullying cases missed (false negatives).")
print(f"  {tn} correctly identified non-bullying cases (true negatives).")


              precision    recall  f1-score   support

Not Bullying       0.99      0.56      0.72       945
    Bullying       0.30      0.97      0.45       178

    accuracy                           0.63      1123
   macro avg       0.64      0.77      0.59      1123
weighted avg       0.88      0.63      0.68      1123

F3 Score for Bullying: 0.7906764168190129
F3 Score for Non-Bullying: 0.5883654058836542
F3 Score Macro: 0.6895209113513335

  1123 cases in total in test set.

  173 out of 178 actual bullying cases identified (true Positives).
  413 false positives (non-bullying identified as bullying).
  5 bullying cases missed (false negatives).
  532 correctly identified non-bullying cases (true negatives).


### Plotting confusion matrix

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt

# plot the confusion matrix of the best threshold
sns.set_theme(style='whitegrid')
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['Predicted 0', 'Predicted 1'], yticklabels=['Actual 0', 'Actual 1'])
plt.xlabel('Predicted Labels')
plt.ylabel('Actual Labels')
plt.title(f'Confusion Matrix, F3-Score ({f3_score_bullying:.2f})')

d:\ludvi\Documents\exjobb\kod\BullyingDetection\.conda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


NameError: name 'conf_matrix' is not defined

<Figure size 576x432 with 0 Axes>